In [1]:
import matplotlib.pyplot as plt
import csv;
import random;
import numpy as np;
import math;
import networkx as nx;

In [2]:
numNet = 10;
for n in range(1,numNet+1):
    G = nx.DiGraph();

    nodes = [10, 10, 10, 10];
    den = [0.33, 0.50, 0.75];
    nEdgesLev = np.zeros(len(den));


    totalNodes = sum(nodes);
    nLayers = len(nodes);

    s = 0;
    t = totalNodes+1;

    lev = 0;
    special = 2; # Level of the special arcs



# Arcs between source and first-layer nodes
    for i in range(1,nodes[0]+1):
        cap = 1000;
        cos = 1000;
        G.add_edge(s, i, capacity=cap, cost=cos, special=0, level=lev);
    
   
# Arcs between Layers

    ctr = 1;
    initNode = 1;
    TotalCap = 0;

    for i in range(1, nLayers):
        lev += 1;
        initNode += nodes[i-1];
        for u in range(ctr, ctr+nodes[i-1]):
            ctr += 1;        
            for v in range(initNode,initNode+nodes[i]):
                I = np.random.binomial(1,den[i-1]);
                if I == 1:
                    if lev == 1:  
                        nEdgesLev[lev-1] += 1;
                        cap = random.randint(nodes[i], 3*nodes[i]);
                        TotalCap += cap;
                        if (lev == special):
                            G.add_edge(u, v, capacity=cap, special=1, level=lev);
                        else:
                            G.add_edge(u, v, capacity=cap, special=0, level=lev);
    
                    else:
                        nEdgesLev[lev-1] += 1;
                        if (lev == special):
                            G.add_edge(u, v, special=1, level=lev);
                        else:
                            G.add_edge(u, v, special=0, level=lev);
                
                       

# Arcs between last-layer and sink
    for i in range(initNode, totalNodes+1):
        cap = 1000;
        cos = 1000;
        G.add_edge(i, t, capacity=cap, cost=cos, special=0, level=lev+1);


# Returning arc
    cap = 10*TotalCap;
    cos = 100000;
    G.add_edge(t, s, capacity=cap, cost=cos, special=0, level=lev+2);
    


# Assigning Capacities

    for u,v in G.edges:
        if (G.edges[u,v]['level'] > 1 and G.edges[u,v]['level'] <= len(den)):
            G.edges[u,v]['capacity'] = random.randint(math.floor(TotalCap/nEdgesLev[G.edges[u,v]['level']-1])-5,
                                                  math.ceil(TotalCap/nEdgesLev[G.edges[u,v]['level']-1]+5));


# Assigning Cost
    for u,v in G.edges:
        if (G.edges[u,v]['level'] == len(den)):
            G.edges[u,v]['cost'] = 1;
        elif (G.edges[u,v]['level'] > 0 and G.edges[u,v]['level'] < len(den)):
            G.edges[u,v]['cost'] = random.randint(math.floor(nEdgesLev[len(den)-1]/nEdgesLev[G.edges[u,v]['level']-1]),
                                              math.ceil(nEdgesLev[len(den)-1]/nEdgesLev[G.edges[u,v]['level']-1]));
    
    
# Writing network file
    myFile = open("Net"+str(n)+".csv","w");
    myFile.write(str(s)+","+str(t)+","+str(special)+"\n");

    for i in range(totalNodes+2):
        for u, v in G.edges:
            if (u == i): 
                myFile.write(str(u)+","+str(v)+","
                    +str(G.edges[u,v]['capacity'])+","
                    +str(G.edges[u,v]['cost'])+","
                    +str(G.edges[u,v]['special'])+","
                    +str(G.edges[u,v]['level'])+"\n");
    
    myFile.close()